# Brain Tumour Detection with Machine Learning
##### Name: Ayat Abdulaziz Gaber Al-Khulaqi 
##### ID: 1191202335

The first step in data preprocessing in machine learning is to acquire the dataset. The dataset used to build and develop the Machine learning models is Brain Tumour Classification (MRI) from the Kaggle website (Brain Tumour Classification (MRI) | Kaggle). Machine learning models used are Transfer Learning and Convolutional Neural Networks (CNN).

In [ ]:
import tensorflow as tf

In [ ]:
# Helper libraries
import os
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
import pathlib
from tensorflow.keras.preprocessing import image

%matplotlib inline

In [ ]:
! unzip data.zip -d BarinMRI

After the dataset was acquired the second step is to import the data into the program. The tf.keras.utils.image_dataset_from_directory generates a tf.data.Dataset from image files in a directory. To create a tf.data.Dataset contains the batch size, image height, image width and object API. Since we have 4 directories in the testing and training folders, num_classes is initialised to 4. For the batch is set to 128 and the image width and height are set to 244. 

To get the object API is needed to use pathlib. To get the tf.data.Dataset is required to use tf.keras.utils.image_dataset_from_directory , then to set the values object API of each directory, image size and batch size. After running the code, it displays the number of files in all directories. 


In [ ]:
num_classes = 4
batch_size = 128
img_height = 224
img_width = 224

In [ ]:
IMAGE_SIZE = (img_height, img_width)

## Data Preperation

In [ ]:
Train_Path = pathlib.Path("/content/BarinMRI/Training")
Test_Path = pathlib.Path("/content/BarinMRI/Testing")

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    Train_Path,
    shuffle=True,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    Test_Path,
    shuffle=True,
    image_size=(img_height, img_width),
    batch_size=batch_size)

#### Class names in the train directory 

In [ ]:
class_names = train_ds.class_names

In [ ]:
print(class_names)

#### Number of classes in train set

In [ ]:
len(class_names)

### Class distribution in training set

Here is the steps needed to display the class distribution for each dataset:
Each dataset is unbatched, which splits the elements of a dataset into multiple elements. The unbatched removes all the batches and it gives all samples one after another.
<br><br>
Then as_numpy_iterator returns an iterator that converts all elements of the dataset to numpy. Using constructor list() to create an empty list, which creates a new list object. Np.array is used to convert a tensor to a numpy array. as_numpy_iterator  used to extract individual samples. Then list() is used to form a list of all individual samples, from the list we form a numpy array. 
<br><br>
Np.unique finds the unique elements of an array.  After the first column of the numpy array which contains the class name  is used in Np.unique. The count is used in the second argument in Np.unique, which counts the number of images in each class.
<br><br>
Lastly, matplotlib.pyplot (plt)  is used to plot the bar graph for each dataset. 


In [ ]:
ds = train_ds.unbatch()
arr = np.array(list(ds.as_numpy_iterator()))

In [ ]:
classes, counts = np.unique(arr[:, 1], return_counts=True)
plt.figure(figsize=((5), (5)))
plt.barh(class_names, counts)
plt.title('Class distribution in training set')

### Class distribution in testing set

In [ ]:
ds2 = test_ds.unbatch()
arr2 = np.array(list(ds2.as_numpy_iterator()))

In [ ]:
classes, counts = np.unique(arr2[:, 1], return_counts=True)
plt.figure(figsize=((5), (5)))
plt.barh(class_names, counts)
plt.title('Class distribution in testing set')

### Displaying Images from Training Directory 

Display the first 25 images from the training and testing set and display the class name below each image. 


In [ ]:
class_names = train_ds.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(25):
    ax = plt.subplot(5, 5, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

### Displaying Images from Testing Directory 

In [ ]:
class_names = test_ds.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(25):
    ax = plt.subplot(5, 5, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

# Convolutional Neural Network(CNN) 

Conventional neural networks (CNN) is a supervised type of Deep learning, mainly used for image and speech recognition. In the sequential model includes: 
rescaling, 2D Convolution, 2D Max Pooling, Flattening and dense layers. 

### Setting up the layers 

In [ ]:
model1 = Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(256, (3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(512, (3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(512, (3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(num_classes)
])

The **2D convolution** block is a layer which can be used to detect spatial features in an image, either working directly on the image data or on the output of previous convolution blocks.  Every block consists of a number of filters, where each filter is a height * width * channels matrix of trainable weights. The filter channel is 3 * 3 ( height by width).
<br><br>
**Flatten** the layer and reshapes the tensor to have a shape that is equal to the number of elements contained in the tensor. 
<br><br>
**2D max pooling** reduces the number of parameters, the size of data, the amount of computation needed, and it controls overfitting. The max pooling block moves a rectangle over the incoming data, selecting the maximum in each certain window. 


### Setting up hyperparameters

In [ ]:
model1.compile(optimizer='adam', 
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])



### Training the model

In [ ]:
history = model1.fit(train_ds, epochs=50,validation_data=(test_ds))

In [ ]:
print("PERFOMANCE",model1.evaluate(test_ds))

### Model Summary

The recalling layer parameter is zero because the layer does not learn anything, however, it rescales and offsets the values of a batch of the image, the rescaling is equal 1./255 as illustrated in sequential model so the  inputs are in the [0, 1] range. To calculate the number of parameters in the 2D convolution layer it needs the input channel number multiplied by kernel height multiplied by kernel width added by 1, and lastly, the final number of parameters is multiplied by the output channels number. 

In [ ]:
model1.summary()

### Plot Training History

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

### Training and Validation Accuracy

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')
plt.show()


### Training and Validation Loss

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('loss')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


# Transfer Learning 

## MobileNetV2 

MobileNetV2 is a convolutional neural network architecture for image classification that was developed by Google.
<br><br>
For transfer learning it’s using tensorflow.keras.applications to import MobileNetV2.  Keras applications are deep learning models that are made available alongside pre-trained weights.  Preprocess input is a tensor or Numpy array encoding a batch of images. 


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

### Augmentation for training set

From tf.keras.preprocessing.image, ImageDataGenerator is used to generate batches of tensor image data with real-time data augmentation. The images are being flipped horizontally and in comments are ways to augment the data.
<br><br>
The Train generator variable it’s using the train data generator, first of coding. However, the test it’s using the test data generator which does not do data augmentation, the second part of coding. Lastly, setting up the train, and test generators, which use the flow from the directory and then provide the directory, target size, color mode, class mode, and batch size.  


In [ ]:
train_data_generator = image.ImageDataGenerator(
                                            horizontal_flip=True,
#                                             zoom_range=0.15,
#                                             width_shift_range=0.2,
#                                             height_shift_range=0.2,
#                                             shear_range=0.15,
                                            preprocessing_function=preprocess_input
                                            )

In [ ]:
# No Augmentation for valid and test
test_data_generator = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_data_generator.flow_from_directory(directory= Train_Path,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'categorical',
                                                    batch_size= batch_size)

test_generator = test_data_generator.flow_from_directory(directory= Test_Path,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'categorical',
                                                    batch_size= batch_size)

### Build The Transfer Model

The base is initialized to MonileNetV2 and it contains weights and includes top args. The weights are a string initialized to imagenet which is pre-training on ImageNet. Include top is a boolean initialized to false to not include the fully-connected layer at the top of the network. Then we set the base to be not trainable, so all children layers become non-trainable as well. The sequential model it’s using the base, global average pooling 2D, dense, and dropout layers.

In [ ]:
base = MobileNetV2(weights='imagenet', include_top=False)
base.trainable = False

model2 = Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(num_classes)
])

The **global average pooling 2D layer** takes a tensor of the size input width * input height * input channels and computes the average value of all values across the whole matrix for each of the input channels. 
<br><br>
The **dense layer** is used to create fully connected layers, in which every output depends on every input
<br><br>
The **dropout layer** randomly sets the outgoing edges of hidden units to 0 at each update of the training phase, which is used to prevent the possibility of  overfitting

### Setting up hyperparameters

Compile the transfer learning model, before that it needs a few more settings :

- Optimizer is an algorithm for adjusting the inner parameters of the model in order to minimise loss.
<br>
- Loss function is an algorithm for measuring how far the model's outputs are from the desired output. The from logits initialised to true indicates that the values of the loss obtained by the model are not normalised, also it is used when we don't have any softmax function in our model.
<br>
- Metrics is used for monitoring the training and testing steps. 


In [ ]:
model2.compile(
    optimizer="Adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

### Training the model

In [ ]:
history2 = model2.fit(train_generator, validation_data=test_generator, epochs=50)
print("PERFOMANCE",model2.evaluate(test_generator))


### Model Summary

In [ ]:
model2.summary()

### Plot Training History

In [ ]:
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']

loss = history2.history['loss']
val_loss = history2.history['val_loss']

### Training and Validation Accuracy

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')
plt.show()

### Training and Validation Loss

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## VGG19

VGG19 is a convolutional neural network model trained on the ImageNet dataset. This model and its variants are used for image classification and other vision tasks. The "19" in the name refers to the number of weight layers in the network. VGG19 consists of 19 weight layers, which include 16 convolutional layers and three fully-connected layers. The convolutional layers are arranged in a series of convolutional blocks. Each block contains two or three convolutional layers and a max pooling layer. The fully-connected layers follow the convolutional layers and are used for classification. 
<br><br>
Using tensorflow.keras.applications to import VGG19 architecture, which is a convolutional neural network that is 19 layers deep.


In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input

In [ ]:
train_data_generator = image.ImageDataGenerator(
                                            horizontal_flip=True,
#                                             zoom_range=0.15,
#                                             width_shift_range=0.2,
#                                             height_shift_range=0.2,
#                                             shear_range=0.15,
                                            preprocessing_function=preprocess_input
                                            )

### Augmentation for training set

In [ ]:
# No Augmentation for valid and test
test_data_generator = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_data_generator.flow_from_directory(directory= Train_Path,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'categorical',
                                                    batch_size= batch_size)

test_generator = test_data_generator.flow_from_directory(directory= Test_Path,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'categorical',
                                                    batch_size= batch_size)

### Build The Transfer Model

In [ ]:
base = VGG19(weights='imagenet', include_top=False)
base.trainable = False

model3 = Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(num_classes)
])

### Setting up hyperparameters

In [ ]:
model3.compile(
    optimizer="Adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

### Training the model

In [ ]:
history3 = model3.fit(train_generator, validation_data=test_generator, epochs=50)
print("PERFOMANCE",model3.evaluate(test_generator))

### Model Summary

In [ ]:
model3.summary()

### Plot Training History

In [ ]:
acc = history3.history['accuracy']
val_acc = history3.history['val_accuracy']

loss = history3.history['loss']
val_loss = history3.history['val_loss']

### Training and Validation Accuracy

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')
plt.show()

### Training and Validation Loss

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## ResNet101V2

ResNet101V2 is an image recognition convolutional neural network trained to distinguish objects in images. It is a subset of the ResNet101 network, which is a more in-depth version of the original ResNet network. The "V2" in the name alludes to an improved version of the original ResNet101 network.



In [ ]:
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input

### Augmentation for training set

In [ ]:
train_data_generator = image.ImageDataGenerator(
                                            horizontal_flip=True,
#                                             zoom_range=0.15,
#                                             width_shift_range=0.2,
#                                             height_shift_range=0.2,
#                                             shear_range=0.15,
                                            preprocessing_function=preprocess_input
                                            )

In [ ]:
# No Augmentation for valid and test
test_data_generator = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_data_generator.flow_from_directory(directory= Train_Path,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'categorical',
                                                    batch_size= batch_size)

test_generator = test_data_generator.flow_from_directory(directory= Test_Path,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'categorical',
                                                    batch_size= batch_size)

### Build The Transfer Model

In [ ]:
base = ResNet101V2(weights='imagenet', include_top=False)
base.trainable = False

model4 = Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(num_classes)
])

### Setting up hyperparameters

In [ ]:
model4.compile(
    optimizer="Adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

### Training the model

In [ ]:
history4 = model4.fit(train_generator, validation_data=test_generator, epochs=50)
print("PERFOMANCE",model4.evaluate(test_generator))

### Model Summary

In [ ]:
model4.summary()

### Plot Training History

In [ ]:
acc = history4.history['accuracy']
val_acc = history4.history['val_accuracy']

loss = history4.history['loss']
val_loss = history4.history['val_loss']

### Training and Validation Accuracy

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')
plt.show()

### Training and Validation Loss

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()